In [24]:
import numpy as np
import plotly.graph_objects as go
from policyengine_us import Simulation
from policyengine_core.charts import *

In [25]:
YEAR = "2024"
MAX_INCOME = 100000

In [26]:
def create_situation(filing_status):
    situation = {
        "people": {
            "adult": {"age": {YEAR: 40}},
            "child_5": {"age": {YEAR: 5}},
            "child_10": {"age": {YEAR: 10}}
        },
        "families": {"family": {"members": ["adult", "child_5", "child_10"]}},
        "marital_units": {"marital_unit": {"members": ["adult"]}},
        "tax_units": {"tax_unit": {"members": ["adult", "child_5", "child_10"]}},
        "households": {
            "household": {"members": ["adult", "child_5", "child_10"], "state_name": {YEAR: "MN"}}
        },
        "axes": [[
            {
                "name": "employment_income",
                "min": 0,
                "max": MAX_INCOME,
                "count": 201,
                "period": YEAR,
            }
        ]]
    }
    
    if filing_status == "married":
        situation["people"]["spouse"] = {"age": {YEAR: 40}}
        for unit in ["families", "marital_units", "tax_units", "households"]:
            situation[unit][list(situation[unit].keys())[0]]["members"].append("spouse")
        
    return situation

In [27]:
def calculate_credits(situation):
    simulation = Simulation(situation=situation)
    wfc = simulation.calculate("mn_wfc", YEAR)
    child_credit = simulation.calculate("mn_child_and_working_families_credits", YEAR)
    return wfc, child_credit

In [34]:
def create_mn_credits_comparison_graph():
    colors = {
        "single_wfc": "#4d94ff",     # Light blue
        "married_wfc": "#0066cc",    # Dark blue
        "single_child": "#999999",   # Light grey
        "married_child": "#333333",  # Dark grey
    }
    x = np.linspace(0, MAX_INCOME, 201)
    fig = go.Figure()

    scenarios = [
        ("single", "Single"),
        ("married", "Married"),
    ]

    for filing_status, label in scenarios:
        wfc, child_credit = calculate_credits(create_situation(filing_status))
        
        fig.add_trace(go.Scatter(
            x=x, 
            y=wfc, 
            mode='lines', 
            name=f"{label} - Working Family Credit (pre HF1938)", 
            line=dict(color=colors[f"{filing_status}_wfc"], dash='solid')
        ))
        
        fig.add_trace(go.Scatter(
            x=x, 
            y=child_credit, 
            mode='lines', 
            name=f"{label} - Child and Working Families Credit (post HF1938)", 
            line=dict(color=colors[f"{filing_status}_child"], dash='solid')
        ))

    # Reorder the traces to match the desired legend order
    fig.data = list(fig.data)
    new_order = [0, 2, 1, 3]  # Single WFC, Married WFC, Single Child, Married Child
    fig.data = [fig.data[i] for i in new_order]

    fig.update_layout(
        title='Minnesota Credits for Households with Children Aged 5 and 10',
        xaxis_title="Employment Income",
        yaxis_title="Credit Amount",
        xaxis=dict(tickformat='$,.0f', range=[0, MAX_INCOME]),
        yaxis=dict(tickformat='$,.0f'),
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.01
        ),
        height=600,
        width=800,
    )
    return fig

In [35]:
# Create and display the chart
fig = create_mn_credits_comparison_graph()
fig = format_fig(fig)
fig.show()